ollama run llama3.2:1b

In [1]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("DSPy")

<Experiment: artifact_location='mlflow-artifacts:/374362034103955121', creation_time=1741686562632, experiment_id='374362034103955121', last_update_time=1741686562632, lifecycle_stage='active', name='DSPy', tags={}>

In [2]:
mlflow.dspy.autolog()

/Users/aidand/dev/hello-dspy/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import dspy

lm = dspy.LM('ollama_chat/llama3.2:1b', api_base='http://localhost:11434')
dspy.configure(lm=lm)

In [5]:
from typing import Literal

class Categorize(dspy.Signature):
    """Classify historic events."""

    event: str = dspy.InputField()
    category: Literal[
        "Wars and Conflicts",
        "Politics and Governance",
        "Science and Innovation",
        "Cultural and Artistic Movements",
        "Exploration and Discovery",
        "Economic Events",
        "Social Movements",
        "Man-Made Disasters and Accidents",
        "Natural Disasters and Climate",
        "Sports and Entertainment",
        "Famous Personalities and Achievements"
    ] = dspy.OutputField()
    confidence: float = dspy.OutputField()

classify = dspy.Predict(Categorize)

# Here is how we call this module
classification = classify(event="World War II[b] or the Second World War (1 September 1939 – 2 September 1945) was a global conflict between two coalitions: the Allies and the Axis powers. Nearly all of the world's countries participated, with many nations mobilising all resources in pursuit of total war. Tanks and aircraft played major roles, enabling the strategic bombing of cities and delivery of the first and only nuclear weapons ever used in war. World War II was the deadliest conflict in history, resulting in 70 to 85 million deaths, more than half of which were civilians. Millions died in genocides, including the Holocaust, and by massacres, starvation, and disease. After the Allied victory, Germany, Austria, Japan, and Korea were occupied, and German and Japanese leaders were tried for war crimes.")
classification

Prediction(
    category='Wars and Conflicts',
    confidence=0.95
)

Trace(request_id=20b8973ecfc34a75aa181fafbd25859a)

In [6]:
classify(event="Second Boer War: In the Battle of Magersfontein the Boers commanded by general Piet Cronjé inflict a defeat on the forces of the British Empire commanded by Lord Methuen trying to relieve the Siege of Kimberley.")

Prediction(
    category='Wars and Conflicts',
    confidence=1.0
)

Trace(request_id=b829509e8a7a4fc491d51125f6329376)

In [18]:
import pandas as pd

with open('events.txt', 'r') as f:
    events = f.readlines()

def classify_event(event_text):
    result = classify(event=event_text)
    return result.category, result.confidence

events = pd.DataFrame(events, columns=['event_string'])

with dspy.context(lm=dspy.LM('ollama_chat/llama3.2:1b', api_base='http://localhost:11434')):

    classifications = events['event_string'].apply(classify_event)

    events['category_32_1b'] = [result[0] for result in classifications]
    events['confidence_32_1b'] = [result[1] for result in classifications]

events

,event_string,category_32_1b,confidence_32_1b
0,627 – Battle of Nineveh: A Byzantine army unde...,Wars and Conflicts,2.00
1,1388 – Maria of Enghien sells the lordship of ...,Wars and Conflicts,1.00
2,1787 – Pennsylvania becomes the second state t...,Politics and Governance,0.95
3,1862 – American Civil War: USS Cairo sinks on ...,Wars and Conflicts,0.50
4,1866 – Oaks explosion: The worst mining disast...,Wars and Conflicts,0.55
5,1870 – Joseph H. Rainey of South Carolina beco...,Politics and Governance,0.50
6,1901 – Guglielmo Marconi receives the first tr...,Science and Innovation,0.95
7,1915 – Yuan Shikai declares the establishment ...,Wars and Conflicts,2.00
8,1917 – Father Edward J. Flanagan founds Boys T...,Wars and Conflicts,2.50
9,"1935 – The Lebensborn Project, a Nazi reproduc...",Wars and Conflicts,0.80


[Trace(request_id=7b7b978572a64a8988351de227994efe), Trace(request_id=a093817ba0c8433ebbc377de1090daf0), Trace(request_id=91155258ceaa47379e39b4e2efbac74f), Trace(request_id=699d3c0b405048b7839b4cbb32409ad1), Trace(request_id=4cbe7a42da0241388e38246825fcb621), Trace(request_id=58b54d03309442719347bbd265000e50), Trace(request_id=43555b02b0f943f384955e4760c8f081), Trace(request_id=c2e3dcccbac94e7698ab0243b522ead5), Trace(request_id=34536ef1d5da4286952de3fea78cfe5e), Trace(request_id=df85c0d8c0e045b6aef5c3275948b175)]

In [19]:
with dspy.context(lm=dspy.LM('fireworks_ai/accounts/fireworks/models/llama-v3p3-70b-instruct')):

    classifications = events['event_string'].apply(classify_event)

    events['category_33_70b'] = [result[0] for result in classifications]
    events['confidence_33_70b'] = [result[1] for result in classifications]

events

,event_string,category_32_1b,confidence_32_1b,category_33_70b,confidence_33_70b
0,627 – Battle of Nineveh: A Byzantine army unde...,Wars and Conflicts,2.00,Wars and Conflicts,0.95
1,1388 – Maria of Enghien sells the lordship of ...,Wars and Conflicts,1.00,Politics and Governance,0.90
2,1787 – Pennsylvania becomes the second state t...,Politics and Governance,0.95,Politics and Governance,0.95
3,1862 – American Civil War: USS Cairo sinks on ...,Wars and Conflicts,0.50,Wars and Conflicts,0.95
4,1866 – Oaks explosion: The worst mining disast...,Wars and Conflicts,0.55,Man-Made Disasters and Accidents,1.00
5,1870 – Joseph H. Rainey of South Carolina beco...,Politics and Governance,0.50,Politics and Governance,0.95
6,1901 – Guglielmo Marconi receives the first tr...,Science and Innovation,0.95,Science and Innovation,0.95
7,1915 – Yuan Shikai declares the establishment ...,Wars and Conflicts,2.00,Politics and Governance,0.95
8,1917 – Father Edward J. Flanagan founds Boys T...,Wars and Conflicts,2.50,Social Movements,0.80
9,"1935 – The Lebensborn Project, a Nazi reproduc...",Wars and Conflicts,0.80,Politics and Governance,0.95


[Trace(request_id=ee151673e1194de1b51108bb21a159b3), Trace(request_id=d4933e369e794b6ca20f47075ca96d8b), Trace(request_id=04c98fdff7ed40a882cdf5714c99495e), Trace(request_id=5cc48772449c4c5694e7a2f67fd64bcc), Trace(request_id=a76a1746296c44a8a8c5f7fc41bbc80c), Trace(request_id=09402910ee1e4f8290a14260774e80d7), Trace(request_id=375802751a3049028944c818b5cec333), Trace(request_id=829fe61fea734318a3acedaa0626e58f), Trace(request_id=657858855d784aa4b7021cb36f8fecac), Trace(request_id=b8b1e50cf87346f39bb77f35b98a2a15)]

In [21]:
# Find rows where the categories disagree
disagreements = events[events['category_32_1b'] != events['category_33_70b']]

# Print each disagreement with both model predictions
for idx, row in disagreements.iterrows():
    print(f"\nEvent: {row['event_string']}")
    print(f"1B Model: {row['category_32_1b']} (confidence: {row['confidence_32_1b']:.2f})")
    print(f"70B Model: {row['category_33_70b']} (confidence: {row['confidence_33_70b']:.2f})")




Event: 1388 – Maria of Enghien sells the lordship of Argos and Nauplia to the Republic of Venice.[2]

1B Model: Wars and Conflicts (confidence: 1.00)
70B Model: Politics and Governance (confidence: 0.90)

Event: 1866 – Oaks explosion: The worst mining disaster in England kills 361 miners and rescuers.[5]

1B Model: Wars and Conflicts (confidence: 0.55)
70B Model: Man-Made Disasters and Accidents (confidence: 1.00)

Event: 1915 – Yuan Shikai declares the establishment of the Empire of China and proclaims himself Emperor.[8]

1B Model: Wars and Conflicts (confidence: 2.00)
70B Model: Politics and Governance (confidence: 0.95)

Event: 1917 – Father Edward J. Flanagan founds Boys Town as a farm village for wayward boys.[9]

1B Model: Wars and Conflicts (confidence: 2.50)
70B Model: Social Movements (confidence: 0.80)

Event: 1935 – The Lebensborn Project, a Nazi reproduction program, is founded by Heinrich Himmler.[10]

1B Model: Wars and Conflicts (confidence: 0.80)
70B Model: Politics a

In [22]:
len(disagreements)

17

In [30]:
def validate_category(example, prediction, trace=None):
    return prediction.category == example.category

In [31]:
cost = sum([x['cost'] for x in lm.history if x['cost'] is not None])  # cost in USD, as calculated by LiteLLM for certain providers
cost

0

In [32]:
# Open and read the 2023 events file
with open('wiki-events-2023.txt', 'r') as f:
    events_2023 = f.readlines()
# Open and read the 2024 events file
with open('wiki-events-2024.txt', 'r') as f:
    events_2024 = f.readlines()

# Combine the events from both years
all_events = events_2023 + events_2024

# Create a new DataFrame with the combined events
all_events_df = pd.DataFrame(all_events, columns=['description'])

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

import json

with dspy.context(lm=dspy.LM('anthropic/claude-3-5-haiku-20241022')):  
    for event in all_events_df['description']:
        result = classify(event=event)
        # Write the result to a jsonl file in append mode
        with open('classifications.jsonl', 'a') as f:
            json.dump({
                'event': event,
                'category': result.category,
                'confidence': result.confidence
            }, f)
            f.write('\n')

    cost = sum([x['cost'] for x in lm.history if x['cost'] is not None])  # cost in USD, as calculated by LiteLLM for certain providers



[Trace(request_id=2823cfdb353744b18bcbddbf353418a9), Trace(request_id=ff42e1e720fe45e4a43e3eca3b649f5f), Trace(request_id=1897ed537f204724aac304417d9a6da7), Trace(request_id=66fdabf0ba9b4149bb1a564b20ddd910), Trace(request_id=acde2d64dea54b99a011f7083a5a4e80), Trace(request_id=7097004e9668410d99b71221850c4ff8), Trace(request_id=19474ad5d42d47f497c38bba74a2b145), Trace(request_id=8dbdb6473c484ffbbbea14709e2872f8), Trace(request_id=0d481764ef9849cabbede2d2d9be9a87), Trace(request_id=5eff710bf6014f60a7783c01ea84cbb1)]

In [33]:
cost

0

In [34]:
# Load the classifications from the JSONL file
classifications = []
with open('classifications.jsonl', 'r') as f:
    for line in f:
        classifications.append(json.loads(line))

# Convert to DataFrame
classifications_df = pd.DataFrame(classifications)

shuffled_df = classifications_df.sample(frac=1, random_state=42)

split_point = int(0.8 * len(shuffled_df))

# Split the data
train_df = shuffled_df.iloc[:split_point].reset_index(drop=True)
test_df = shuffled_df.iloc[split_point:].reset_index(drop=True)

# Reset indices
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

print(f"Training set size: {len(train_df)}")
print(f"Testing set size: {len(test_df)}")

Training set size: 390
Testing set size: 98


In [37]:
test_df

,event,category,confidence
0,2024 Comorian presidential election: Amid an o...,Politics and Governance,0.95
1,November 27 – After forming a coalition Govern...,Politics and Governance,0.95
2,December 12 – At the COP28 climate summit in D...,Politics and Governance,0.95
3,June 6–9 – The 2024 European Parliament electi...,Politics and Governance,0.95
4,November 29 – 2024 Irish general election: Cen...,Politics and Governance,0.95
...,...,...,...
93,Russian invasion of Ukraine: The Nova Kakhovka...,Wars and Conflicts,0.95
94,April 1 – Israel attacks the Iranian embassy i...,Wars and Conflicts,0.95
95,\n,Politics and Governance,0.50
96,French-Algerian writer Boualem Sansal is put i...,Politics and Governance,0.85


In [45]:
from dspy import Evaluate

testset = [dspy.Example(event=row['event'], category=row['category']).with_inputs("event") for _, row in test_df.iterrows()]
evaluator = Evaluate(devset=testset, num_threads=1, display_progress=True, display_table=5)
evaluator(classify, metric=validate_category)

Average Metric: 33.00 / 98 (33.7%): 100%|██████████| 98/98 [01:16<00:00,  1.28it/s]

2025/03/13 16:45:28 INFO dspy.evaluate.evaluate: Average Metric: 33 / 98 (33.7%)


,event,example_category,pred_category,confidence,validate_category
0,"2024 Comorian presidential election: Amid an opposition boycott, i...",Politics and Governance,Politics and Governance,0.62,✔️ [True]
1,November 27 – After forming a coalition Government with the right-...,Politics and Governance,Wars and Conflicts,0.10,
2,"December 12 – At the COP28 climate summit in Dubai, a consensus is...",Politics and Governance,Economic Events,0.50,
3,"June 6–9 – The 2024 European Parliament election is held. The EPP,...",Politics and Governance,Wars and Conflicts,1.00,
4,November 29 – 2024 Irish general election: Centre-right party Fian...,Politics and Governance,Wars and Conflicts,2.50,


33.67

[Trace(request_id=3ab32a21ccab420491e4715ce5df3a29), Trace(request_id=5de91f0bc40246c89f56e240ed29c13d), Trace(request_id=f106f80c529b4b3e8538afa327071c9b), Trace(request_id=09c3bbf5352649d6b07fbbc3119adc79), Trace(request_id=640c9113c4ad48c2aa79205ba412217f), Trace(request_id=e728316d99544e879280e3fcd79f99e5), Trace(request_id=02fd7f5b63c840268bd3f78ad69c519c), Trace(request_id=3e0881a3f8a84a4c9cb0c2a2a9b41092), Trace(request_id=46423c4ce5a74e6fa6af5f48069bf53c), Trace(request_id=7a09ff40334f42d89fb169c7f1cb47a0)]

In [46]:
trainset = [dspy.Example(event=row['event'], category=row['category']).with_inputs("event") for _, row in train_df.iterrows()]

lm = dspy.LM('ollama_chat/llama3.2:1b', api_base='http://localhost:11434')
dspy.configure(lm=lm)

# Optimize
tp = dspy.MIPROv2(metric=validate_category, auto="light")
optimized_classify = tp.compile(classify, trainset=trainset, max_labeled_demos=0, max_bootstrapped_demos=0)

2025/03/13 16:46:35 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: True
num_candidates: 7
valset size: 100

2025/03/13 16:46:37 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/03/13 16:46:37 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used for informing instruction proposal.

2025/03/13 16:46:37 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=7 sets of demonstrations...


Bootstrapping set 1/7
Bootstrapping set 2/7


 17%|█▋        | 13/78 [00:08<00:40,  1.61it/s]


Bootstrapped 3 full traces after 13 examples for up to 1 rounds, amounting to 13 attempts.
Bootstrapping set 3/7


 15%|█▌        | 12/78 [00:06<00:38,  1.73it/s]


...

2025/03/13 16:52:28 INFO dspy.evaluate.evaluate: Average Metric: 18 / 25 (72.0%)
2025/03/13 16:52:28 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 72.0 on minibatch of size 25 with parameters ['Predictor 0: Instruction 4'].
2025/03/13 16:52:28 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [72.0, 56.0, 72.0, 68.0, 72.0]
2025/03/13 16:52:28 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [26.0]
2025/03/13 16:52:28 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 26.0
2025/03/13 16:52:28 INFO dspy.teleprompt.mipro_optimizer_v2: ========================================


2025/03/13 16:52:28 INFO dspy.teleprompt.mipro_optimizer_v2: == Trial 7 / 8 - Minibatch ==



Average Metric: 14.00 / 25 (56.0%): 100%|██████████| 25/25 [00:09<00:00,  2.72it/s]

2025/03/13 16:52:37 INFO dspy.evaluate.evaluate: Average Metric: 14 / 25 (56.0%)
2025/03/13 16:52:37 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 56.0 on minibatch of size 25 with parameters ['Predictor 0: Instruction 1'].
2025/03/13 16:52:37 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [72.0, 56.0, 72.0, 68.0, 72.0, 56.0]
2025/03/13 16:52:37 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [26.0]
2025/03/13 16:52:37 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 26.0
2025/03/13 16:52:37 INFO dspy.teleprompt.mipro_optimizer_v2: ========================================


2025/03/13 16:52:37 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 8 / 8 - Full Evaluation =====
2025/03/13 16:52:37 INFO dspy.teleprompt.mipro_optimizer_v2: Doing full eval on next top averaging program (Avg Score: 72.0) from minibatch trials...



Average Metric: 68.00 / 100 (68.0%): 100%|██████████| 100/100 [02:28<00:00,  1.49s/it]

2025/03/13 16:55:06 INFO dspy.evaluate.evaluate: Average Metric: 68 / 100 (68.0%)
2025/03/13 16:55:06 INFO dspy.teleprompt.mipro_optimizer_v2: New best full eval score! Score: 68.0
2025/03/13 16:55:06 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [26.0, 68.0]
2025/03/13 16:55:06 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 68.0
2025/03/13 16:55:06 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/03/13 16:55:06 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/03/13 16:55:06 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 68.0!


In [47]:
evaluator(optimized_classify, metric=validate_category)

Average Metric: 70.00 / 98 (71.4%): 100%|██████████| 98/98 [06:42<00:00,  4.11s/it]

2025/03/13 17:06:32 INFO dspy.evaluate.evaluate: Average Metric: 70 / 98 (71.4%)


,event,example_category,pred_category,confidence,validate_category
0,"2024 Comorian presidential election: Amid an opposition boycott, i...",Politics and Governance,Politics and Governance,0.8,✔️ [True]
1,November 27 – After forming a coalition Government with the right-...,Politics and Governance,Politics and Governance,0.8,✔️ [True]
2,"December 12 – At the COP28 climate summit in Dubai, a consensus is...",Politics and Governance,Economic Events,0.8,
3,"June 6–9 – The 2024 European Parliament election is held. The EPP,...",Politics and Governance,Politics and Governance,0.8,✔️ [True]
4,November 29 – 2024 Irish general election: Centre-right party Fian...,Politics and Governance,Politics and Governance,0.8,✔️ [True]


71.43

[Trace(request_id=1c2f06aff78a453bb2f5661c88cdd589), Trace(request_id=6cbda2edc44844689931cdaeafe31287), Trace(request_id=dd5e5fa84d9a46978fbfd9f1a4c23dc8), Trace(request_id=1cced7301baa422889e03947efd92b75), Trace(request_id=1327677978cf4b00979d269541e370f2), Trace(request_id=ba8cd3b0cde04e7ebb4963ed23ac4325), Trace(request_id=b509d618fb99479b8f900ba31c60cae7), Trace(request_id=69be3eb7cc494be3b32b711ab245dde9), Trace(request_id=e7e4590bbe71478495c577e623dd59b4), Trace(request_id=324487467e4c44a1ad97149f8d96832d)]

In [ ]:
lm.inspect_history(n=1)

In [49]:
optimized_classify(event="Maria of Enghien sells the lordship of Argos and Nauplia to the Republic of Venice.")

Prediction(
    category='Politics and Governance',
    confidence=0.8
)

Trace(request_id=b5174fffef1c4751b2cee1417a43c836)

In [50]:
def classify_event_optimized(event_text):
    result = optimized_classify(event=event_text)
    return result.category, result.confidence

classifications = events['event_string'].apply(classify_event_optimized)

events['category_31_1b_optimized'] = [result[0] for result in classifications]
events['confidence_31_1b_optimized'] = [result[1] for result in classifications]

events

,event_string,category_32_1b,confidence_32_1b,category_33_70b,confidence_33_70b,category_31_1b_optimized,confidence_31_1b_optimized
0,627 – Battle of Nineveh: A Byzantine army under Emperor Heraclius ...,Wars and Conflicts,2.00,Wars and Conflicts,0.95,Wars and Conflicts,0.8
1,1388 – Maria of Enghien sells the lordship of Argos and Nauplia to...,Wars and Conflicts,1.00,Politics and Governance,0.90,Politics and Governance,0.8
2,1787 – Pennsylvania becomes the second state to ratify the US Cons...,Politics and Governance,0.95,Politics and Governance,0.95,Politics and Governance,0.9
3,1862 – American Civil War: USS Cairo sinks on the Yazoo River.[4]\n,Wars and Conflicts,0.50,Wars and Conflicts,0.95,Wars and Conflicts,0.8
4,1866 – Oaks explosion: The worst mining disaster in England kills ...,Wars and Conflicts,0.55,Man-Made Disasters and Accidents,1.00,Wars and Conflicts,0.9
5,1870 – Joseph H. Rainey of South Carolina becomes the second black...,Politics and Governance,0.50,Politics and Governance,0.95,Politics and Governance,0.8
6,1901 – Guglielmo Marconi receives the first transatlantic radio si...,Science and Innovation,0.95,Science and Innovation,0.95,Science and Innovation,0.9
7,1915 – Yuan Shikai declares the establishment of the Empire of Chi...,Wars and Conflicts,2.00,Politics and Governance,0.95,Politics and Governance,0.8
8,1917 – Father Edward J. Flanagan founds Boys Town as a farm villag...,Wars and Conflicts,2.50,Social Movements,0.80,Politics and Governance,0.9
9,"1935 – The Lebensborn Project, a Nazi reproduction program, is fou...",Wars and Conflicts,0.80,Politics and Governance,0.95,Politics and Governance,0.8


[Trace(request_id=2cd9eeb613274a4b9da9542efea738cc), Trace(request_id=fcc42c7338bb498090421dc9f7eb34d1), Trace(request_id=2b5d7cc6794243d48ab563c83d7b74d5), Trace(request_id=bee9553122514f348c5c52984b87f3a1), Trace(request_id=0ec48e5f09414b6e9a1a1dd817867b9c), Trace(request_id=2752845ecb8c43239a5cab9e3b6756d3), Trace(request_id=dddd154d625e4f12bf6bec7283ed8acc), Trace(request_id=fe9f4e248d8543a792e91abf9c2b9679), Trace(request_id=31c12e1d96724bb1a10f3eafcaff07b2), Trace(request_id=3a530d44f5124e5d8ac991f3c9ee94e7)]

In [51]:
# Save optimized model
optimized_classify.save("optimized_classify_31_1b.json")

In [54]:
disagreements = events[events['category_31_1b_optimized'] != events['category_33_70b']]
# len(disagreements)

print("old: ", (len(events) - 17)/len(events))
print("new: ", (len(events) - len(disagreements))/len(events))

old:  0.46875
new:  0.625


In [55]:
from dspy.teleprompt import *

# Load our model
lm = dspy.LM('ollama_chat/llama3.2:1b', api_base='http://localhost:11434')
prompt_gen_lm = dspy.LM('fireworks_ai/accounts/fireworks/models/llama-v3p3-70b-instruct')
dspy.configure(lm=lm)

# Optimize
tp = dspy.MIPROv2(metric=validate_category, auto="light", prompt_model=prompt_gen_lm, task_model=lm)
optimized_classify = tp.compile(classify, trainset=trainset, max_labeled_demos=0, max_bootstrapped_demos=0)

2025/03/13 17:16:12 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: True
num_candidates: 7
valset size: 100

2025/03/13 17:16:13 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/03/13 17:16:13 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used for informing instruction proposal.

2025/03/13 17:16:13 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=7 sets of demonstrations...


Bootstrapping set 1/7
Bootstrapping set 2/7


 17%|█▋        | 13/78 [00:00<00:00, 556.69it/s]


Bootstrapped 3 full traces after 13 examples for up to 1 rounds, amounting to 13 attempts.
Bootstrapping set 3/7


 15%|█▌        | 12/78 [00:00<00:00, 755.42it/s]


...

2025/03/13 17:21:00 INFO dspy.evaluate.evaluate: Average Metric: 15 / 25 (60.0%)
2025/03/13 17:21:00 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 60.0 on minibatch of size 25 with parameters ['Predictor 0: Instruction 4'].
2025/03/13 17:21:00 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [60.0, 52.0, 60.0, 56.0, 60.0]
2025/03/13 17:21:00 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [26.0]
2025/03/13 17:21:00 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 26.0
2025/03/13 17:21:00 INFO dspy.teleprompt.mipro_optimizer_v2: ========================================


2025/03/13 17:21:00 INFO dspy.teleprompt.mipro_optimizer_v2: == Trial 7 / 8 - Minibatch ==



Average Metric: 13.00 / 25 (52.0%): 100%|██████████| 25/25 [00:23<00:00,  1.08it/s]

2025/03/13 17:21:23 INFO dspy.evaluate.evaluate: Average Metric: 13 / 25 (52.0%)
2025/03/13 17:21:23 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 52.0 on minibatch of size 25 with parameters ['Predictor 0: Instruction 1'].
2025/03/13 17:21:23 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [60.0, 52.0, 60.0, 56.0, 60.0, 52.0]
2025/03/13 17:21:23 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [26.0]
2025/03/13 17:21:23 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 26.0
2025/03/13 17:21:23 INFO dspy.teleprompt.mipro_optimizer_v2: ========================================


2025/03/13 17:21:23 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 8 / 8 - Full Evaluation =====
2025/03/13 17:21:23 INFO dspy.teleprompt.mipro_optimizer_v2: Doing full eval on next top averaging program (Avg Score: 60.0) from minibatch trials...



Average Metric: 62.00 / 100 (62.0%): 100%|██████████| 100/100 [02:05<00:00,  1.25s/it]

2025/03/13 17:23:28 INFO dspy.evaluate.evaluate: Average Metric: 62 / 100 (62.0%)
2025/03/13 17:23:28 INFO dspy.teleprompt.mipro_optimizer_v2: New best full eval score! Score: 62.0
2025/03/13 17:23:28 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [26.0, 62.0]
2025/03/13 17:23:28 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 62.0
2025/03/13 17:23:28 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/03/13 17:23:28 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/03/13 17:23:28 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 62.0!


In [56]:
optimized_classify.save("optimized_classify_31_1b_2.json")

In [57]:
evaluator(optimized_classify, metric=validate_category)

Average Metric: 62.00 / 98 (63.3%): 100%|██████████| 98/98 [05:34<00:00,  3.42s/it]

2025/03/13 17:31:56 INFO dspy.evaluate.evaluate: Average Metric: 62 / 98 (63.3%)


,event,example_category,pred_category,confidence,validate_category
0,"2024 Comorian presidential election: Amid an opposition boycott, i...",Politics and Governance,Politics and Governance,0.8,✔️ [True]
1,November 27 – After forming a coalition Government with the right-...,Politics and Governance,Politics and Governance,0.8,✔️ [True]
2,"December 12 – At the COP28 climate summit in Dubai, a consensus is...",Politics and Governance,Politics and Governance,0.8,✔️ [True]
3,"June 6–9 – The 2024 European Parliament election is held. The EPP,...",Politics and Governance,Politics and Governance,0.8,✔️ [True]
4,November 29 – 2024 Irish general election: Centre-right party Fian...,Politics and Governance,Politics and Governance,0.8,✔️ [True]


63.27

[Trace(request_id=fefaad5587d0426ea946f5add78ee357), Trace(request_id=16ce8d707e0e47ac8aebbf394f483aca), Trace(request_id=60f2d37d8eaf4abea7ac38fc8846f88d), Trace(request_id=adea65d5ea3148f7a603a3da17d37a9f), Trace(request_id=c7bc87e4d7af4c2d8942faf01afe0fbd), Trace(request_id=b45ca52fe397496d96b8f0ce8329d427), Trace(request_id=1da6c28839a540fa9b27eb703febcc6c), Trace(request_id=c5272603aaab4aecacc494ab48aac143), Trace(request_id=34af1ff67ffe4fcd8912fa9007f794a1), Trace(request_id=891e5c4f398e481095c79a934042794a)]

In [ ]:
classifications = events['event_string'].apply(classify_event_optimized)

events['category_31_1b_optimized_2'] = [result[0] for result in classifications]
events['confidence_31_1b_optimized_2'] = [result[1] for result in classifications]

events